In [1]:
# file reading/writing 
import storage 
import csv

# analysis 
import networkx as nx
from haversine import haversine

# standard plotting 
import seaborn 
import matplotlib.pyplot as plt

# mapping 
import folium
from IPython.display import display, IFrame
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
from shapely.geometry import shape, Point


# standard utility
import numpy as np
import random
import json
import pandas as pd

# text processing
import spacy


In [2]:
departments = storage.retrieve_all_departments()
article_dept_links = storage.get_article_department_links()
print(departments[0:5])

[(1, 'Department of Neurosurgery, Stanford University School of Medicine, Stanford, CA 94305, USA.', 37.424669, -122.170275), (2, 'Department of Chemistry, Stanford University, Stanford, CA 94305, USA.', 37.424669, -122.170275), (3, 'Department of Chemistry, Stanford University, Stanford, CA 94305, USA.', 37.424669, -122.170275), (4, 'Department of Neurosurgery, Stanford University School of Medicine, Stanford, CA 94305, USA.', 37.424669, -122.170275), (5, 'Department of Neurosurgery, Stanford University School of Medicine, Stanford, CA 94305, USA.', 37.424669, -122.170275)]


Save this to a JSONL for annotation

In [3]:
all_dep_list = []
for i in departments:
    all_dep_list.append({"listed": i[1], "annotated_name":""})
with open('dep_for_annotation.jsonl', 'w') as file:
    json.dump(all_dep_list, file)
    

Load annotated university names

In [4]:
from io import StringIO
with open('annotated_university_names.txt') as f:
    annotated_names = json.load(f)
    annotated_names_dataframe = pd.json_normalize(annotated_names)
annotated_names_dataframe.head()


,listed,annotated_name
0,"Department of Neurosurgery, Stanford Universit...",Stanford University
1,"Department of Chemistry, Stanford University, ...",Stanford University
2,"Department of Chemistry, Stanford University, ...",Stanford University
3,"Department of Neurosurgery, Stanford Universit...",Stanford University
4,"Department of Neurosurgery, Stanford Universit...",Stanford University


In [5]:
print(annotated_names_dataframe.describe())

                                                listed  \
count                                              682   
unique                                             392   
top     Regeneron Genetics Center, Tarrytown, NY, USA.   
freq                                                31   

                 annotated_name  
count                       682  
unique                      259  
top     University of Melbourne  
freq                         33  


## Loading list of universities 

Utilizing [this public domain Kaggle dataset by *The Devastator*](https://www.kaggle.com/datasets/thedevastator/all-universities-in-the-world?resource=download) the institutions available in our scraped dataset can be resolved to a known and fixed university.

In [6]:
countries_df = pd.read_csv("world-universities.csv", header=None)
countries_df.head()

,0,1,2
0,AD,University of Andorra,http://www.uda.ad/
1,AE,Abu Dhabi University,http://www.adu.ac.ae/
2,AE,Ajman University of Science & Technology,http://www.ajman.ac.ae/
3,AE,Alain University of Science and Technology,http://www.alainuniversity.ac.ae/
4,AE,Al Ghurair University,http://www.agu.ae/


It's best to rename our columns to something more descriptive.

In [7]:
countries_df.columns = ['countrycode', 'name', 'website']  
countries_df.head()
countries_df.insert(0, "annotated", annotated_names_dataframe['annotated_name'])

We would like to disambiguate the mentions of universities to a unique reference point to ensure our graph has the correct number of nodes. 
As such, we will make use of the functionalities of the [spAcy EntityLinker](https://spacy.io/api/entitylinker).

First, load a spacy model

In [8]:
nlp = spacy.load("en_core_web_sm")

Next, we create a Spacy knowledge base. This will serve as a repository of entities which may be resolved to.

In [9]:
from spacy.kb import InMemoryLookupKB
kb = InMemoryLookupKB(vocab=nlp.vocab, entity_vector_length=96)

In [10]:
#addToKB args: pipeline, knowledgebase, ID, name
dict_strict_name = {}
def addToKB(nlp, kb, ID, e_name, alias):
    print(e_name)
    if(e_name is not pd.NA):
        document_name = nlp(e_name)
        name_vector = document_name.vector
        #add entity
        
        kb.add_entity(entity=str(e_name), entity_vector=name_vector, freq=666)
        #add alias which ensures a resolution to our entity
        kb.add_alias(alias=alias, entities=[str(e_name)], probabilities=[1])
    return(kb)

In [14]:
countries_df = annotated_names_dataframe.dropna()
# this could be rewritten using df.apply 
for row, col in countries_df.iterrows():
    kb = addToKB(nlp, kb, row, col['annotated_name'],col['listed'])

Stanford University
Stanford University
Stanford University
Stanford University
Stanford University
Stanford University
Stanford University
Stanford University
Chan Zuckerberg Biohub
Stanford University
Stanford University
University Hospital, Bern, Switzerland
Emory University
Stanford University
Stanford University
Stanford University
Stanford University
Stanford University
Chan Zuckerberg Biohub
Chan Zuckerberg Biohub
Chan Zuckerberg Biohub
Oregon Health and Science University
Stanford University
Stanford University
Stanford University
Stanford University
Stanford University
University of Melbourne
St. Jude Children's Research Hospital
University of Melbourne
La Trobe University
St. Jude Children's Research Hospital
Fudan University
Fudan University


/tmp/ipykernel_19155/2712039995.py:10: UserWarning: [W018] Entity 'Stanford University' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=str(e_name), entity_vector=name_vector, freq=666)
/tmp/ipykernel_19155/2712039995.py:12: UserWarning: [W017] Alias 'Department of Chemistry, Stanford University, Stanford, CA 94305, USA.' already exists in the Knowledge Base.
  kb.add_alias(alias=alias, entities=[str(e_name)], probabilities=[1])
/tmp/ipykernel_19155/2712039995.py:12: UserWarning: [W017] Alias 'Department of Neurosurgery, Stanford University School of Medicine, Stanford, CA 94305, USA.' already exists in the Knowledge Base.
  kb.add_alias(alias=alias, entities=[str(e_name)], probabilities=[1])
/tmp/ipykernel_19155/2712039995.py:12: UserWarning: [W017] Alias 'Department of Anesthesiology, Perioperative & Pain Medicine, Stanford University School of Medicine, Stanford, CA 94305, USA.' already exists in the Knowledge Base.
  kb.add_alias(alias=al

Harvard Medical School
Harvard Medical School
University of Melbourne
University of Melbourne
University of Melbourne
University of Melbourne
University of Melbourne
University of Melbourne
University of Melbourne
University of Melbourne
University of Melbourne
University of Melbourne
University of Melbourne
Monash University
Walter Eliza Hall Institute of Medical Research
University of Melbourne
University of Melbourne
University of Melbourne
University of Melbourne
University of Córdoba
University of Tennessee
St. Jude Children's Research Hospital
University of California, Santa Cruz
St. Jude Children's Research Hospital
University of Melbourne
University of Melbourne
University of Melbourne
University of Melbourne
University of Melbourne
University of Melbourne
University of Tasmania
University of Melbourne
Walter Eliza Hall Institute of Medical Research
St. Jude Children's Research Hospital
University of Tennessee
Monash University
Imperial College London
Imperial College London
Un

/tmp/ipykernel_19155/2712039995.py:10: UserWarning: [W018] Entity 'Harvard Medical School' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=str(e_name), entity_vector=name_vector, freq=666)
/tmp/ipykernel_19155/2712039995.py:12: UserWarning: [W017] Alias 'Melbourne Mass Spectrometry and Proteomics Facility, Bio21 Molecular Science and Biotechnology Institute, University of Melbourne, Parkville, VIC 3052, Australia.' already exists in the Knowledge Base.
  kb.add_alias(alias=alias, entities=[str(e_name)], probabilities=[1])
/tmp/ipykernel_19155/2712039995.py:12: UserWarning: [W017] Alias 'Department of HostMicrobe Interactions, St. Jude Children's Research Hospital, Memphis, TN 38105, USA.' already exists in the Knowledge Base.
  kb.add_alias(alias=alias, entities=[str(e_name)], probabilities=[1])
/tmp/ipykernel_19155/2712039995.py:10: UserWarning: [W018] Entity 'Walter Eliza Hall Institute of Medical Research' already exists in the Knowledge B

Fudan University
University of Melbourne
University of Melbourne
University of Melbourne
The Netherlands Cancer Institute
The Netherlands Cancer Institute
The Netherlands Cancer Institute
The Netherlands Cancer Institute
The Netherlands Cancer Institute
The Netherlands Cancer Institute
Leiden University
The Netherlands Cancer Institute
The Netherlands Cancer Institute
The Netherlands Cancer Institute
The Netherlands Cancer Institute
Leiden University
Amsterdam University
Amsterdam University
Leiden University
Leiden University
The Netherlands Cancer Institute
H. Lee Moffitt Cancer Center
Leiden University
McGill University
The Netherlands Cancer Institute
Leiden University
Leiden University
The Netherlands Cancer Institute
University of California, San Diego
University of California, San Francisco
University of California, San Diego
University of California, San Francisco
University of California, San Diego
University of California, San Diego
University of California, San Diego
Peking 

...

George Washington University
Global Action for Trans Equality, New York, NY
Chicago Department of Public Health, Chicago
Blantyre City Council, Blantyre, Malawi.
Georgetown University
Positive Voice, Greece, Athens.
Stop TB Partnership, Geneva
World Health Organization
International Association of Providers of AIDS Care, Washington, DC, USA.
Ministry of Health, City of Buenos Aires, Buenos Aires, Argentina.
Montefiore University
American University
Cameroonian Association for the Development and Empowerment of Vulnerable People, Yaoundé, Cameroon.
Mayor's Office of Health Policy, New Orleans, LA, USA.
University of Barcelona
Aidshilfe Wien
Chile Positivo
Fenway Institute
The Well Project, Miami, FL, USA.
Fundación México Vivo, Mexico City, Mexico.
University of Milan
General Directorate of Public Health, Region of Madrid, Madrid, Spain.
FAQ Health, Hamburg, Germany.
UN Human Settlements Programme, Nairobi, Kenya.
Nairobi City County Department of Health
APCOM Foundation, Bangkok, Thail

/tmp/ipykernel_19155/2712039995.py:10: UserWarning: [W018] Entity 'University of Antananarivo' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=str(e_name), entity_vector=name_vector, freq=666)
/tmp/ipykernel_19155/2712039995.py:12: UserWarning: [W017] Alias 'Faculty of Medicine, University of Antananarivo, Antananarivo, Madagascar' already exists in the Knowledge Base.
  kb.add_alias(alias=alias, entities=[str(e_name)], probabilities=[1])
/tmp/ipykernel_19155/2712039995.py:10: UserWarning: [W018] Entity 'Kaiser Permanente, San Francisco' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=str(e_name), entity_vector=name_vector, freq=666)
/tmp/ipykernel_19155/2712039995.py:10: UserWarning: [W018] Entity 'University of Miami' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=str(e_name), entity_vector=name_vector, freq=666)
/tmp/ipykernel_19155/2712039995.py:10: U

Kaiser Permanente, San Francisco
University of Miami
Ruane Clinical Research, Los Angeles
University of Paris
Men's Health Foundation, Los Angeles, CA, USA.
Central Texas Clinical Research, Austin, TX, USA.
Midway Research Center, Fort Pierce, FL, USA.
Royal Sussex County Hospital
The Crofoot Research Centre, Houston, TX
The Research Institute, Springfield, MA
Gilead Sciences, Foster City, CA, USA. 
Gilead Sciences, Foster City, CA, USA. 
Gilead Sciences, Foster City, CA, USA. 
Gilead Sciences, Foster City, CA, USA. 
Gilead Sciences, Foster City, CA, USA. 
Gilead Sciences, Foster City, CA, USA. 
Gilead Sciences, Foster City, CA, USA. 
University of California San Francisco
Philadelphia FIGHT Community Health Centres, Philadelphia, PA, USA.
The Fenway Institute, Fenway Health
US Agency for International Development
Moi University
Newlands Clinic, Harare, Zimbabwe.
US Centers for Disease Control and Prevention
Newlands Clinic, Harare, Zimbabwe.
Moi University
Kilimanjaro Christian Medica

/tmp/ipykernel_19155/2712039995.py:12: UserWarning: [W017] Alias 'Department of Epidemiology and Biostatistics, University of California San Francisco, San Francisco, CA, USA.' already exists in the Knowledge Base.
  kb.add_alias(alias=alias, entities=[str(e_name)], probabilities=[1])
/tmp/ipykernel_19155/2712039995.py:10: UserWarning: [W018] Entity 'University of Cape Town' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=str(e_name), entity_vector=name_vector, freq=666)
/tmp/ipykernel_19155/2712039995.py:12: UserWarning: [W017] Alias 'Division of HIV, Infectious Diseases, and Global Medicine, Department of Medicine, University of California San Francisco, San Francisco, CA, USA.' already exists in the Knowledge Base.
  kb.add_alias(alias=alias, entities=[str(e_name)], probabilities=[1])
/tmp/ipykernel_19155/2712039995.py:12: UserWarning: [W017] Alias 'Center for Clinical Research, Kenya Medical Research Institute, Thika, Kenya' already exist

ValueError: [E141] Entity vectors should be of length 96 instead of the provided 0.

In [15]:
#print(f"Entities in the KB: {kb.get_entity_strings()}")
print(f"Aliases in the KB: {kb.get_alias_strings()}")

Aliases in the KB: ['Department of Bioengineering and Therapeutic Sciences, University of California, San Francisco, San Francisco, CA 94158, USA', 'International Association of Providers of AIDS Care, Washington, DC, USA.', "Center for Infectious Diseases Research, St. Jude Children's Research Hospital, Memphis, TN 38105, USA", 'Department of Microbiology and Immunology, University of Melbourne, Peter Doherty Institute for Infection and Immunity, Melbourne, VIC 3000, Australia. ', 'Department of Anatomy and Physiology, University of Melbourne, Parkville, VIC 3010, Australia.', 'School of Public Health, University of California, Berkeley, CA, USA.', 'Institute of Medical Biochemistry, Center for Molecular Biology of Inflammation (ZMBE), University of Münster, Münster 48149, Germany.', 'Institute for Cancer Genetics, Department of Pediatrics, and Department of Genetics and Development, Columbia University Irving Medical Center, New York, NY 10032, USA.', 'National AIDS, STIs and Hepatit

In [17]:
print(f"Candidates for Boston University': {[c.alias_ for c in kb.get_alias_candidates('Boston University')]}")


Candidates for Boston University': []
